Setup
First off, check that the Python environment you installed in the readme is running this notebook. Make sure you select the py38 kernel in the top right of this notebook. You should see a 3.8 version when you run this command.

In [1]:
import sys
sys.version

'3.8.19 | packaged by conda-forge | (default, Mar 20 2024, 12:47:35) \n[GCC 12.3.0]'

Next we need to install some libraries.

In [2]:
%pip install --user "google-cloud-aiplatform>=1.25.0" --upgrade
%pip install --user "google-cloud-aiplatform[pipelines]>=1.25.0"
%pip install --user "langchain>=0.0.216"
%pip install --user webvtt-py webvtt-to-json requests
%pip install --user graphdatascience
%pip install --user pydantic
%pip install --user IProgress
%pip install --user tqdm
%pip install --user neo4j_tools
%pip install --user gradio
%pip install --user gcsfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 1.0 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 2.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.7/333.7 kB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 2.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 2.7 M

Now restart the kernel.

That will allow the Python evironment to import the new packages.

In [3]:
pip install gcsfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 1.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os._exit(0)

In [6]:
# Note, you will need to set your project_id
project_id = 'neo4j-field-engineering-apac'
LOCATION = 'us-central1' # SHould be us-central1 to make sure you have all features from VertexAI

In [7]:
import vertexai
vertexai.init(project=project_id, location=LOCATION)

In [8]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from graphdatascience import GraphDataScience
from neo4j_tools import gds_db_load, gds_utils
import gradio as gr

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

Data Ingestion
You will need a Neo4j AuraDS Pro instance. You can deploy that on Google Cloud Marketplace here.

With that complete, you'll need to install the Neo4j library and set up your database connection.

Before loading the data, create constraints as below

Get Data, Create Constraints, and Load
Before loading data into Neo4j, it is usually best practice to create Key or Uniqueness constraints for nodes. These constraints act as an index with some validation on unique id properties and thus make MATCH statements run significantly faster. Not doing this can result in a VERY slow ingest, so this is a critical step.

We will be using convenience functions for loading nodes and relationships in batch. As the data load runs, you can see the Cypher statements being used.

In [21]:
import pandas as pd
from neo4j import GraphDatabase
import gcsfs

# Define the connection to the Neo4j database
uri = "neo4j+s://e138db66.databases.neo4j.io"  # Update with your Neo4j URI
user = "neo4j"  # Update with your Neo4j username
password = "WQ-_qT98eDNvLtHjjjFfbiO3Jn7JvmLD-dAA-5_kv0Y"  # Update with your Neo4j password

# Create the Neo4j driver
driver = GraphDatabase.driver(uri, auth=(user, password))

# Load data from CSV files
fs = gcsfs.GCSFileSystem()

with fs.open('gs://yanyee-petronas-employee-bucket-test/employee_data.csv') as f:
    employee_df = pd.read_csv(f)

with fs.open('gs://yanyee-petronas-employee-bucket-test/training_and_development_data.csv') as f:
    training_df = pd.read_csv(f)

def create_constraints(tx):
    tx.run('CREATE CONSTRAINT unique_employee_id IF NOT EXISTS FOR (n:Employee) REQUIRE n.EmpID IS UNIQUE')
    tx.run('CREATE CONSTRAINT unique_program_name IF NOT EXISTS FOR (n:Program) REQUIRE n.ProgramName IS UNIQUE')

# Create constraints
with driver.session() as session:
    session.execute_write(create_constraints)

def load_data_apoc(query, params):
    with driver.session() as session:
        session.run(query, parameters=params)

# Prepare queries
employee_query = """
CALL apoc.periodic.iterate(
    "UNWIND $rows AS row RETURN row",
    "MERGE (e:Employee {
        EmpID: row.EmpID, 
        FirstName: row.FirstName, 
        LastName: row.LastName, 
        StartDate: toDate(row.StartDate), 
        Title: row.Title, 
        Supervisor: row.Supervisor, 
        Email: row.ADEmail, 
        BusinessUnit: row.BusinessUnit, 
        EmployeeType: row.EmployeeType, 
        DepartmentType: row.DepartmentType, 
        Division: row.Division, 
        DOB: row.DOB, 
        State: row.State, 
        JobFunctionDescription: row.JobFunctionDescription, 
        GenderCode: row.GenderCode, 
        MaritalDesc: row.MaritalDesc
    })",
    {batchSize: 500, params: {rows: $rows}}
)
"""

program_query = """
CALL apoc.periodic.iterate(
    "UNWIND $rows AS row RETURN row",
    "MERGE (p:Program {ProgramName: row.ProgramName})",
    {batchSize: 10, params: {rows: $rows}}
)
"""

training_query = """
CALL apoc.periodic.iterate(
    "UNWIND $rows AS row RETURN row",
    "MATCH (e:Employee {EmpID: row.EmployeeID})
     MATCH (p:Program {ProgramName: row.TrainingProgramName})
     MERGE (e)-[:TAKEN_TRAINING {
        TrainingDate: row.TrainingDate,
        TrainingType: row.TrainingType,
        TrainingOutcome: row.TrainingOutcome,
        Location: row.Location,
        Trainer: row.Trainer,
        TrainingDuration: row.TrainingDurationDays,
        TrainingCost: row.TrainingCost
    }]->(p)",
    {batchSize: 500, params: {rows: $rows}}
)
"""

# Convert DataFrames to dictionaries
employee_data = employee_df.to_dict('records')
training_data = training_df.to_dict('records')

# Extract unique programs
unique_programs = training_df[['TrainingProgramName']].drop_duplicates().rename(columns={'TrainingProgramName': 'ProgramName'}).to_dict('records')

# Load Employee nodes using APOC
load_data_apoc(employee_query, {'rows': employee_data})

# Load Program nodes using APOC
load_data_apoc(program_query, {'rows': unique_programs})

# Load TAKEN_TRAINING relationships using APOC
load_data_apoc(training_query, {'rows': training_data})

# Close the driver
driver.close()


Start of chatbot

In [12]:
pip install --user langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.llms import VertexAI
# from langchain_google_vertexai import VertexAI
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
1. Generate Cypher query compatible ONLY for Neo4j Version 5
2. Do not use EXISTS, SIZE keywords in the cypher. Use alias when using the WITH keyword
3. Use only Nodes and relationships mentioned in the schema
4. Always enclose the Cypher output inside 3 backticks
5. Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Company name use `toLower(c.name) contains 'neo4j'`
6. Candidate node is synonymous to Person
7. Always use aliases to refer the node in the query
8. Cypher is NOT SQL. So, do not mix and match the syntaxes
Schema:
{schema}
Samples:
Question: Who are the employees who have taken Customer Service program but has not completed?
Answer: MATCH (e:Employee)-[t:TAKEN_TRAINING]->(p:Program)
WHERE toLower(p.ProgramName) CONTAINS 'customer service' AND t.TrainingOutcome <> 'Completed'
RETURN e.FirstName, e.LastName;
Answer:
"""
@retry(tries=5, delay=5)
def get_results(messages):
    start = timer()
    try:
        graph = Neo4jGraph(
            url=host, 
            username=user, 
            password=password
        )
        chain = GraphCypherQAChain.from_llm(
            VertexAI(
                    model_name=codey_model_name,
                    max_output_tokens=2048,
                    temperature=0,
                    top_p=0.95,
                    top_k=0.40), 
            graph=graph, verbose=True,
            return_intermediate_steps=True,
            cypher_prompt=CYPHER_GENERATION_PROMPT
        )
        if messages:
            question = messages.pop()
        else: 
            question = 'How many employees are there?'
        return chain(question)
    finally:
        print('Cypher Generation Time : {}'.format(timer() - start))

In [15]:
pip install -U langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 824.0 kB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
r = chain("""What are the programs available?""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Program)
RETURN p.ProgramName

Full Context:
[{'p.ProgramName': 'Customer Service'}, {'p.ProgramName': 'Leadership Development'}, {'p.ProgramName': 'Technical Skills'}, {'p.ProgramName': 'Communication Skills'}, {'p.ProgramName': 'Project Management'}]

> Finished chain.


In [17]:
print(f"Intermediate steps: {r['intermediate_steps']}")
print(f"Final answer: {r['result']}")

Intermediate steps: [{'query': 'cypher\nMATCH (p:Program)\nRETURN p.ProgramName\n'}, {'context': [{'p.ProgramName': 'Customer Service'}, {'p.ProgramName': 'Leadership Development'}, {'p.ProgramName': 'Technical Skills'}, {'p.ProgramName': 'Communication Skills'}, {'p.ProgramName': 'Project Management'}]}]
Final answer:  The available programs are: Customer Service, Leadership Development, Technical Skills, Communication Skills, Project Management.


In [4]:
r = chain("""How many people failed the Customer Service program?""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e:Employee)-[t:TAKEN_TRAINING]->(p:Program)
WHERE toLower(p.ProgramName) CONTAINS 'customer service' AND t.TrainingOutcome = 'Failed'
RETURN count(DISTINCT e)

Full Context:
[{'count(DISTINCT e)': 136}]

> Finished chain.


In [5]:
r = chain("""Who joined the company before 2020?""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e:Employee)
WHERE e.StartDate < '2020-01-01'
RETURN e.FirstName, e.LastName;

Full Context:
[{'e.FirstName': 'Uriah', 'e.LastName': 'Bridges'}, {'e.FirstName': 'Paula', 'e.LastName': 'Small'}, {'e.FirstName': 'Edward', 'e.LastName': 'Buck'}, {'e.FirstName': 'Maruk', 'e.LastName': 'Fraval'}, {'e.FirstName': 'Latia', 'e.LastName': 'Costa'}, {'e.FirstName': 'Sharlene', 'e.LastName': 'Terry'}, {'e.FirstName': 'Jac', 'e.LastName': 'McKinzie'}, {'e.FirstName': 'Myriam', 'e.LastName': 'Givens'}, {'e.FirstName': 'Dheepa', 'e.LastName': 'Nguyen'}, {'e.FirstName': 'Xana', 'e.LastName': 'Potts'}]

> Finished chain.


In [19]:
r = chain("""How many people joined the company before 2023?""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e:Employee)
WHERE e.StartDate < '2023'
RETURN count(e)



Failed to write data to connection ResolvedIPv4Address(('52.44.11.98', 7687)) (ResolvedIPv4Address(('52.44.11.98', 7687)))
Failed to write data to connection IPv4Address(('p-e138db66-cd06-0003.production-orch-0032.neo4j.io', 7687)) (ResolvedIPv4Address(('50.16.105.214', 7687)))


Full Context:
[{'count(e)': 1948}]

> Finished chain.
